# Init Notebook

In [ ]:
# Import necessary libraries
from dotenv import load_dotenv
from openai import OpenAI
import os
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm

from embeddings_comparison.utils.llm_clients.cached_client import CachedLLMClient
from embeddings_comparison.utils.llm_clients.open_ai_client import OPEN_AI_MODEL_NAME, OpenAIClient


# Load environment variables from .env file
load_dotenv()

open_ai_key = os.environ["OPEN_AI_KEY"]

openai_client = OpenAIClient(api_key=open_ai_key, model_name=OPEN_AI_MODEL_NAME.GPT_4O)
cached_client = CachedLLMClient(client=openai_client)

# Data curation
Downloading set of articles to be used for assessment

In [ ]:
from embeddings_comparison.utils import wiki_parser
from pprint import pprint
from embeddings_comparison.utils.storage import ArticleStorage

storage = ArticleStorage()

raw_pages = wiki_parser.extract_pages_from_file("data/Wikipedia-20241111162837.xml")
storage.save_articles(raw_pages)

# Data filtering

Remove sections that don't contain text or are not relevant

In [ ]:
pages_df = storage.load_all()
pages_df = pages_df[pages_df['Section Title'] != 'Linki zewnętrzne']
pages_df = pages_df[pages_df['Section Title'] != 'Zobacz też']
pages_df = pages_df[pages_df['Section Title'] != 'Bibliografia']
pages_df

# Augment data

In [ ]:
from embeddings_comparison.utils.question_generation import BASE_PROMT_PL, generate_question_for_text
from tqdm.autonotebook import tqdm

pbar = tqdm(total=len(pages_df))
questions_column = []
for _, row in pages_df.iterrows():
    pbar.set_description(f"Generating questions for {row['Section Title']}")
    questions = generate_question_for_text(cached_client, row['Section Content'], BASE_PROMT_PL)
    questions_column.append(questions.questions)
    pbar.update(1)

pages_df['questions'] = questions_column

# Data Ingestion

In [ ]:
from embeddings_comparison.utils.embedding_models.caching import CachedEmbeddingModel
from embeddings_comparison.utils.embedding_models.hugging_face import HF_EMBEDDING_MODEL_NAME, HFEmbeddingModel
from embeddings_comparison.utils.embedding_models.open_ai import OPENAI_EMBEDDING_MODEL_NAME, OpenAIEmbeddingModel
from embeddings_comparison.utils.vectordb.vectordb import VectorDB, VectorIndex
from tqdm.autonotebook import tqdm

vector_db = VectorDB()
vector_db.add_index(f"OPENAI_SMALL", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_3_SMALL)))
vector_db.add_index(f"OPENAI_LARGE", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_3_LARGE)))
vector_db.add_index(f"OPENAI__ADA", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_ADA_002)))
vector_db.add_index("HF_SDADAS", CachedEmbeddingModel(HFEmbeddingModel(model_name=HF_EMBEDDING_MODEL_NAME.ST_POLISH_PARAPHRASE_FROM_DISTILROBERTA)))

for index_name in tqdm(vector_db.list_indices(), desc='Testing Embedding models'):
    vector_db.insert_texts(pages_df['Section Content'].values.tolist(), index_name)

# Perform the experiment

In [ ]:
from tqdm.autonotebook import tqdm

pbar = tqdm(total=len(vector_db.list_indices()))
pbar.update(1)
for index_name in vector_db.list_indices():
    pbar.set_description(f"Testing {index_name}")
    identified_matches = []
    for _, row in tqdm(pages_df.iterrows(), desc=f"Testing rows for {index_name}"):
        matched_with_answer = 0
        for question in row['questions']:
            found_text = vector_db.find_text(text=question, top_k=1, index_name=index_name)[0]

            if found_text == row['Section Content']:
                matched_with_answer += 1
        identified_matches.append(matched_with_answer)
    pages_df[index_name] = identified_matches
    pbar.update(1)